In [166]:
import pandas as pd
import numpy as np
import os, sys
from datetime import date
from bs4 import BeautifulSoup
import requests

pd.set_option("display.max_columns", 100)

In [2]:
def p(s):
    print(s)
    
pd.DataFrame.len = pd.Index.len = lambda x: print(len(x))

In [156]:
# SEASON DATA
def getSeasonData():
    prefix = {
        'Passing':'Pass',
        'Rushing':'Rush',
        'Receiving':'Rec',
        'Off.':'Off',
        'Def.':'Def',
        'ST.':'ST',
        'Fantasy':'Fan',
        'Scoring':'Tot'
    }

    df = pd.DataFrame()
    years = np.arange(1970,2020)
    for year in years:
        sys.stdout.write(f'{year} \r'); sys.stdout.flush()

        # See if possible to read from local file
        path = f'../data/pfr/{year}.csv'
        if os.path.exists(path):
            dfYear = pd.read_csv(path, index_col=0)
        else:        
            # Fetch data from web
            url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
            dfYear = pd.read_html(url)[0]

            # Remove MultiIndex and disambiguate cols
            cols = dfYear.columns.get_level_values(-1)
            newCols = []
            for i, col in enumerate(cols):
                if col in cols[cols.duplicated()]:
                    topLevel = dfYear.columns.get_level_values(-2)[i]
                    newCol = prefix.get(topLevel,'ERROR')+col
                else:
                    newCol = col

                newCols.append(newCol)
            dfYear.columns = newCols

            # Cleanup
            dfYear['Season'] = year
            dfYear = dfYear[dfYear.Rk != 'Rk']
            dfYear = dfYear.apply(pd.to_numeric,errors='ignore')

            # Save to local file
            dfYear.to_csv(path)

        df = df.append(dfYear)

    if not os.path.exists('../data/pfr/all.csv'):
        df.to_csv('../data/pfr/all.csv')
        
    print('Done.')
        
    return df

In [168]:
path = '../data/pfr/all.csv'
if os.path.exists(path):
    df = pd.read_csv(path,index_col=0)
else:
    df = getSeasonData()
    
df = df.reset_index(drop=True)
df = df[df.Rk != 'Rk']
df = df.apply(pd.to_numeric,errors='ignore')
df.insert(df.columns.get_loc('PPR'),'HalfPPR',(df['FantPt'] + df['PPR'])/2)
df

C:\Users\mrkawa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (35,36,37,38,39,40,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,PassAtt,PassYds,PassTD,Int,RushAtt,RushYds,Y/A,RushTD,Rec,RecYds,Y/R,RecTD,Fmb,FL,TotTD,2PM,2PP,FantPt,HalfPPR,PPR,DKPt,FDPt,VBD,PosRank,OvRank,Season,Tgt,passAtt,passYds,passTD,rushAtt,rushYds,rushTD,recYds,recTD,totTD
0,1,Ron Johnson*+,NYG,RB,23,14.0,14.0,0.0,0.0,0.0,0.0,0.0,263.0,1027.0,3.90,8.0,48.0,487.0,10.15,4.0,5.0,NaN,12.0,NaN,NaN,223.0,244.70,266.4,274.9,242.4,136.0,1,1.0,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Dick Gordon*+,CHI,WR,26,14.0,14.0,0.0,0.0,0.0,0.0,0.0,4.0,17.0,4.25,0.0,71.0,1026.0,14.45,13.0,2.0,NaN,13.0,NaN,NaN,182.0,216.65,251.3,255.3,215.8,134.0,1,2.0,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Gene Washington*+,SFO,WR,23,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,53.0,1100.0,20.75,12.0,0.0,NaN,12.0,NaN,NaN,182.0,208.50,235.0,238.0,208.5,134.0,2,3.0,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Gary Garrison*,SDG,WR,26,14.0,14.0,0.0,0.0,0.0,0.0,0.0,4.0,7.0,1.75,0.0,44.0,1006.0,22.86,12.0,0.0,NaN,12.0,NaN,NaN,173.0,195.15,217.3,220.3,195.3,124.0,3,4.0,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,MacArthur Lane *,STL,RB,28,14.0,14.0,0.0,0.0,0.0,0.0,0.0,206.0,977.0,4.74,11.0,32.0,365.0,11.41,2.0,2.0,NaN,13.0,NaN,NaN,212.0,227.10,242.2,249.2,226.2,123.0,2,5.0,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,616,Ray-Ray McCloud,CAR,NaN,23,6.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,-2.0,-2.00,-2.0,-1.0,-2.0,NaN,242,NaN,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26971,617,Darrius Shepherd,GNB,WR,24,6.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,NaN,1.00,NaN,1.0,1.0,NaN,NaN,NaN,-2.0,-1.45,-0.9,0.1,-1.4,NaN,241,NaN,2019,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
26972,618,Jarrett Stidham,NWE,QB,23,3.0,0.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,-1.00,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,-2.0,-1.80,-1.6,-0.6,-0.6,NaN,73,NaN,2019,0.0,4.0,14.0,0.0,2.0,-2.0,0.0,0.0,0.0,0.0
26973,619,Michael Walker,JAX,WR,23,7.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2.0,NaN,7.50,NaN,2.0,2.0,NaN,NaN,NaN,-3.0,-1.75,-0.5,1.5,-1.5,NaN,245,NaN,2019,3.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0


In [171]:
df.Tm

0        NYG
1        CHI
2        SFO
3        SDG
4        STL
        ... 
26970    CAR
26971    GNB
26972    NWE
26973    JAX
26974    PHI
Name: Tm, Length: 26128, dtype: object

In [53]:
# GAME DATA
url = 'https://www.pro-football-reference.com'
year = 2019
maxp = 5
    
# grab fantasy players
r = requests.get(url + '/years/' + str(year) + '/fantasy.htm')
soup = BeautifulSoup(r.content, 'html.parser')
parsed_table = soup.find_all('table')[0]  

df = pd.DataFrame()

# first 2 rows are col headers
for i,row in enumerate(parsed_table.find_all('tr')[2:]):
    sys.stdout.write(f'{i}/{maxp} \r'); sys.stdout.flush()
    if i >= maxp: 
        break
        
    try:
        dat = row.find('td', attrs={'data-stat': 'player'})
        name = dat.a.get_text()
        stub = dat.a.get('href')
        stub = stub[:-4] + '/fantasy/' + str(year)
        pos = row.find('td', attrs={'data-stat': 'fantasy_pos'}).get_text()

        # grab this players stats
        tdf = pd.read_html(url + stub)[0]

        # get rid of MultiIndex, just keep last row
        tdf.columns = tdf.columns.get_level_values(-1)
        
        tdf = tdf.rename(columns={'Unnamed: 4_level_2': 'Away'})
        tdf['Away'] = [1 if r=='@' else 0 for r in tdf['Away']]

        # drop all intermediate stats
        tdf = tdf.iloc[:,[1,2,3,4,5,-3]]
        
        # drop "Total" row
        tdf = tdf.query('Date != "Total"')
        
        # add other info
        tdf['Name'] = name
        tdf['Position'] = pos
        tdf['Season'] = year

        df = df.append(tdf)
    except:
        raise

print('\nComplete.')

5/5 
Complete.
